In [1]:
library(tidyverse)
library(caret)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




# Predicting Tennis Match Winners

# Introduction

The US Open grand slam tennis tournament has an estimated total prize money pool of USD $50.4 million. This puts it in line with high-value tournaments such as the MLB World Series and the PGA FedEx Cup (source: pledgesports.org). With such a high potential payoff, there is much incentive for competitive tennis athletes and coaches to understand the player attributes that contribute to tournament success. 

This project will study the data of tennis match results collected for the top 500 tennis players. The purpose of the study is to determine whether certain attributes to greater competitive success by analyzing player and match information. The dataset contains results from nearly 7,000 matches and includes the winner and loser heights, playing hands, countries of origin, ages, match times and other information (source: https://github.com/JeffSackmann/tennis_atp). 

# Preliminary Exploratory Data Analysis

In [2]:
data2017 <- read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2017.csv')
data2018 <- read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2018.csv')
data2019 <- read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2019.csv')

combined_data <- rbind(data2017, data2018, data2019)

tennis <- combined_data %>% 
    select(winner_hand, loser_hand, winner_age, loser_age, winner_ht, loser_ht, minutes, w_ace, l_ace)

tennis_win = select(tennis, winner_hand, winner_age, winner_ht, minutes, w_ace)
tennis_loss = select(tennis, loser_hand, loser_age, loser_ht, minutes, l_ace)

tennis_win = mutate(tennis_win, hand = winner_hand, age = winner_age, height = winner_ht, aces = w_ace, result = 'w') %>%
            select(-winner_hand, -winner_age, -winner_ht, -w_ace)
tennis_loss = mutate(tennis_loss, hand = loser_hand, age = loser_age, height = loser_ht, aces = l_ace, result = 'l') %>%
            select(-loser_hand, -loser_age, -loser_ht, -l_ace)

cleaned_data = rbind(tennis_win, tennis_loss)

cleaned_data = mutate(cleaned_data, result = as.factor(result))

head(cleaned_data)

training_rows <- cleaned_data %>%
    select(result) %>% 
    unlist() %>%
    createDataPartition (p = 0.75 , list = FALSE)

training_set <- tennis %>% slice (training_rows)
testing_set <- tennis %>% slice (-training_rows)

Parsed with column specification:
cols(
  .default = col_double(),
  tourney_id = col_character(),
  tourney_name = col_character(),
  surface = col_character(),
  tourney_level = col_character(),
  winner_entry = col_character(),
  winner_name = col_character(),
  winner_hand = col_character(),
  winner_ioc = col_character(),
  loser_entry = col_character(),
  loser_name = col_character(),
  loser_hand = col_character(),
  loser_ioc = col_character(),
  score = col_character(),
  round = col_character()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  .default = col_double(),
  tourney_id = col_character(),
  tourney_name = col_character(),
  surface = col_character(),
  tourney_level = col_character(),
  winner_entry = col_character(),
  winner_name = col_character(),
  winner_hand = col_character(),
  winner_ioc = col_character(),
  loser_entry = col_character(),
  loser_name = col_character(),
  loser_hand = col_character(),
  loser_ioc = 

minutes,hand,age,height,aces,result
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<fct>
91,R,30.89391,193,11,w
130,R,21.98494,NA,11,w
125,R,22.85832,185,7,w
75,R,20.23272,NA,12,w
90,R,22.70500,NA,1,w
83,R,34.75428,175,3,w


# Methods

In order to wrangle our data, we have to use the read_csv() function on the raw links to the data on: https://github.com/JeffSackmann/tennis_atp. Because we are looking across three years of data we read 3 files and combine them using the rbind() function. Cleaning the data involved separating the winners and losers, creating a new result column, and then recombining them into one cleaned data set.

For classification analysis we will be using the following columns:

* item winner_hand (the dominant hand of the winner)
* item loser_hand (the dominant hand of the loser)
* item winner_age 
* item loser_age
* item winner_ht (the height of the winner)
* item loser_ht (the height of the loser)
* item minutes (The length of the match)
* item w_ace (number of aces the winner had)
* item l_ace (the number aces the loser had)

and creating a knn-model in order to classify a whether or not a hypothetical player with given attributes will be able to win their match. We will be removing the left over columns from the set as they are not variables of interest for this analysis.

A visualisation we will create is a plot of height versus aces and colour the points by whether they were wins or losses.

# Expected outcomes and significance